In [13]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol="AAPl,msft",
    interval="1d",
    start_date="2021-01-01",
    end_date="2021-01-01"
)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
toolbox

Toolbox(symbol=AAPL,MSFT, interval=1d, start_date=2021-01-01, end_date=2021-01-01, provider=yfinance)

In [20]:
result = toolbox.technical.mandelbrot_channel()

In [21]:
result

date,open,high,low,close,volume,split_ratio,dividend,symbol
date,f64,f64,f64,f64,i64,f64,f64,str
2021-01-04,133.520004,133.610001,126.760002,129.410004,143301900,0.0,0.0,"""AAPL"""
2021-01-04,222.529999,223.0,214.809998,217.690002,37130100,0.0,0.0,"""MSFT"""
2021-01-05,128.889999,131.740005,128.429993,131.009995,97664900,0.0,0.0,"""AAPL"""
2021-01-05,217.259995,218.520004,215.699997,217.899994,23823000,0.0,0.0,"""MSFT"""
2021-01-06,127.720001,131.050003,126.379997,126.599998,155088000,0.0,0.0,"""AAPL"""
…,…,…,…,…,…,…,…,…
2024-03-13,418.100006,418.179993,411.450012,415.100006,17115900,0.0,0.0,"""MSFT"""
2024-03-14,172.910004,174.309998,172.050003,173.0,72913500,0.0,0.0,"""AAPL"""
2024-03-14,420.23999,427.820007,417.98999,425.220001,34157300,0.0,0.0,"""MSFT"""


## Test validation

In [12]:
from openbb import obb

context_params = toolbox

equity_historical_data = (
    obb.equity.price.historical(
        symbol="AAPL,MSFT",
        start_date=context_params.start_date,
        end_date=context_params.end_date,
        provider=context_params.provider,
    ).to_polars()
).drop(["dividends", "stock_splits"])


In [ ]:
print(equity_historical_data.dtypes)
print(equity_historical_data.columns)


In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.toolbox import ToolboxData


ToolboxData(equity_historical_data)



## Reprex

In [ ]:
from typing import Optional
import pandera.polars as pa
import polars as pl


class ModelWithChecks(pa.DataFrameModel):
    a: int
    b: str
    c: Optional[float]
    date: pl.Date


valid_lf = pl.LazyFrame(
    {
        "a": pl.Series([1, 2, 3], dtype=pl.Int64),
        "b": ["d", "e", "f"],
        # "c": [0.0, 0.6, 1],
        "date": pl.date_range(
            start=datetime.date(2021, 1, 1),
            end=datetime.date(2021, 1, 3),
            eager=True,
        ),
    }
)


ModelWithChecks.validate(valid_lf, lazy=True).collect()
